## Importing Important Libraries
### Steps To Be Followed

    Importing necessary Libraries
    Creating S3 bucket
    Mapping train And Test Data in S3
    Mapping The path of the models in S3

In [20]:
import os
import boto3
import pandas as pd
import sagemaker
from sagemaker.inputs import TrainingInput

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [17]:
bucket_name = "llamamodelsfortranslation"
my_region =boto3.session.Session().region_name
print(my_region)

eu-west-2


In [9]:
import boto3

s3 = boto3.client('s3', region_name='eu-west-2')

s3.create_bucket(
    Bucket='llamamodelsfortranslation',
    CreateBucketConfiguration={
        'LocationConstraint': 'eu-west-2'
    }
)

{'ResponseMetadata': {'RequestId': 'X3CVCDRGJR3ZR483',
  'HostId': '8LYhIonYNwvKSet9NxcZglCgNI3J/X071hqj9APsmciBKvG8+X+qmP7MhqEI1RJ4B9bAilCDd1o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8LYhIonYNwvKSet9NxcZglCgNI3J/X071hqj9APsmciBKvG8+X+qmP7MhqEI1RJ4B9bAilCDd1o=',
   'x-amz-request-id': 'X3CVCDRGJR3ZR483',
   'date': 'Thu, 05 Sep 2024 20:02:48 GMT',
   'location': 'http://llamamodelsfortranslation.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://llamamodelsfortranslation.s3.amazonaws.com/'}

In [21]:
prefix = "sample"
output_path = "s3://{}/{}/train".format(bucket_name ,prefix)
print(output_path)

s3://llamamodelsfortranslation/sample/train


## Downloading The Dataset And Storing in S3

In [13]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [14]:

### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [23]:
# Combine and save the training data as CSV
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Upload the CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Define S3 input for SageMaker (replacing the deprecated s3_input)
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train/train.csv'.format(bucket_name, prefix), content_type='csv')

# Now s3_input_train can be used for training in SageMaker

In [24]:
# Combine and save the testing data as CSV
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

# Upload the CSV file to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Define S3 input for SageMaker (replacing the deprecated s3_input)
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test/test.csv'.format(bucket_name, prefix), content_type='csv')

# Now s3_input_test can be used for testing in SageMaker

## Building Models Xgboot- Inbuilt Algorithm

In [25]:
import sagemaker

# Use the newer `sagemaker.image_uris.retrieve` method
container = sagemaker.image_uris.retrieve(
    framework='xgboost', 
    region=boto3.Session().region_name, 
    version='1.0-1'
)

print(f'XGBoost container URI: {container}')


XGBoost container URI: 764974769150.dkr.ecr.eu-west-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


In [26]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [27]:
# Construct a SageMaker estimator using the updated SageMaker SDK
estimator = sagemaker.estimator.Estimator(
    image_uri=container,  # Changed from image_name to image_uri
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,  
    instance_type='ml.m5.2xlarge',  
    volume_size=5,  
    output_path=output_path,
    use_spot_instances=True,  
    max_run=300, 
    max_wait=600  
)

print("Estimator created successfully!")

Estimator created successfully!


In [28]:
# Assuming s3_input_train and s3_input_test are already defined using TrainingInput
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

print("Training started!")

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-05-20-49-01-674


2024-09-05 20:49:07 Starting - Starting the training job...
2024-09-05 20:49:23 Starting - Preparing the instances for training...
2024-09-05 20:49:58 Downloading - Downloading the training image...
2024-09-05 20:50:34 Training - Training image download completed. Training in progress...[2024-09-05 20:50:50.996 ip-10-0-185-35.eu-west-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single no

## Deploy Machine Learning Model As Endpoints

In [30]:
# Deploy the model to a SageMaker endpoint
xgb_predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge'
)

print("Model deployed successfully!")


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-05-20-54-26-404
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-09-05-20-54-26-404
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-09-05-20-54-26-404


-----!Model deployed successfully!


## Prediction of the Test Data

In [31]:
import numpy as np
from sagemaker.serializers import CSVSerializer

# Prepare test data by removing unwanted columns
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Configure the predictor
xgb_predictor.content_type = 'text/csv'  # Set content type for inference
xgb_predictor.serializer = CSVSerializer()  # Use CSVSerializer from the new SageMaker SDK

# Make predictions
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')

# Convert predictions from string to a numpy array
predictions_array = np.fromstring(predictions[1:], sep=',')

# Print the shape of the predictions array
print(predictions_array.shape)


(12357,)


In [33]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [36]:
import pandas as pd
import numpy as np

# Create the confusion matrix
cm = pd.crosstab(index=test_data['y_yes'], 
                 columns=np.round(predictions_array), 
                 rownames=['Observed'], 
                 colnames=['Predicted'])

# Extract true positives, false negatives, true negatives, and false positives
tn = cm.iloc[0, 0]
fn = cm.iloc[1, 0]
tp = cm.iloc[1, 1]
fp = cm.iloc[0, 1]

# Calculate the overall classification rate
accuracy = (tp + tn) / (tp + tn + fp + fn) * 100

# Print the results
print("\n{0:<25}{1:<4.1f}%\n".format("Overall Classification Rate: ", accuracy))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format(
    "No Purchase", tn / (tn + fn) * 100, tn, fp / (tp + fp) * 100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<})\n".format(
    "Purchase", fn / (tn + fn) * 100, fn, tp / (tp + fp) * 100, tp))



Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297)



## Deleting The Endpoints

In [37]:
# Import necessary libraries
import boto3
import sagemaker

# Delete the SageMaker endpoint
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)
# Delete all objects in the S3 bucket
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.delete()
print("Endpoint and bucket contents deleted successfully!")

INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-09-05-20-54-26-404


Endpoint and bucket contents deleted successfully!
